6- El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta información se encuentra guardada en un dataframe con el siguiente formato: (fecha, id_caso, descripcion, estado_caso, categoria, latitud, longitud). Los posibles estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolución. Las fechas se encuentran en el formato YYYY-MM-DD. Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un Dataframe con el siguiente formato (id_caso, respuesta), siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él El sector encargado de las estadísticas oficiales del GCPD quiere con esta información analizar las siguientes situaciones: 
    A. Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no participó Batman).
    B. Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos en los que fue llamado con la batiseñal, participó en la resolución). 
    C. Indicar el mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import datetime

In [2]:
tam=1000
def crearDescripcion(x):
    listaTipos=['robo','asesinato','violacion','bandalismo','incendio']
    listaReturn=[]
    for i in range(x):
        listaReturn.append(listaTipos[random.randint(0,len(listaTipos)-1)])
    return listaReturn

def crearEstadoCaso(x):
    listaTipos=['caso abierto', 'caso resuelto', 'cerrado sin resolución']
    listaReturn=[]
    for i in range(x):
        listaReturn.append(listaTipos[random.randint(0,len(listaTipos)-1)])
    return listaReturn

def random_date(start, end, tiempo):
    """Generate a random datetime between `start` and `end`"""
    x = start + datetime.timedelta(seconds=random.randint(0, int((end - start).total_seconds())),
    ) 
    dicc = {'dia': x.day,'mes': x.month,'anio':x.year,'fecha':x.date()}
    return dicc[tiempo]

f=[]
for x in range (tam): 
    f.append(random_date(pd.to_datetime('1/1/2018'),pd.to_datetime('1/1/2020'),'fecha'))

gotham = {'fecha':f, 
            'id_caso':list(range(100,100+tam)), 
            'descripcion':crearDescripcion(tam), 
            'estado_caso': crearEstadoCaso(tam), 
            'categoria':np.random.randint(1,6,size=tam),
            'latitud':np.random.randint(100,400,size=tam), 
            'longitud':np.random.randint(3000,5000,size=tam)}
gdf = pd.DataFrame(gotham)

batman = {'id_caso':list(range(100,100+tam)),
         'respuesta':np.random.randint(0,2,size=tam)}
bdf=pd.DataFrame(batman)
bdf.head()

,id_caso,respuesta
0,100,0
1,101,0
2,102,0
3,103,0
4,104,1


A. Tasa de resolución de casos de la fuerza policial por categoría de caso (considerando aquellos casos en los que no participó Batman)

In [3]:
ddf = gdf.merge(bdf,on='id_caso')
ddf

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud,respuesta
0,2018-01-08,100,asesinato,caso resuelto,1,212,4998,0
1,2018-05-23,101,bandalismo,caso resuelto,4,179,4521,0
2,2018-11-30,102,asesinato,caso resuelto,2,157,3973,0
3,2018-08-17,103,bandalismo,cerrado sin resolución,5,356,4069,0
4,2018-01-08,104,violacion,caso resuelto,5,327,3541,1
...,...,...,...,...,...,...,...,...
995,2018-05-11,1095,bandalismo,cerrado sin resolución,1,139,4859,0
996,2019-01-20,1096,asesinato,caso resuelto,2,127,3943,1
997,2018-07-10,1097,robo,cerrado sin resolución,3,168,3105,1
998,2018-12-19,1098,violacion,caso abierto,4,325,3727,0


In [4]:
tasaRes = len(ddf[(ddf['respuesta']==0) & (ddf['estado_caso']=='caso resuelto')]
             )/len(ddf[(ddf['respuesta']==0) & (ddf['estado_caso']!='caso abierto')])*100
print ('Son resueltos %f %% de los casos' %tasaRes)

Son resueltos 50.310559 % de los casos


B. Tasa de resolución de casos con la ayuda de Batman (considerando que aquellos casos en los que fue llamado con la batiseñal, participó en la resolución).

In [5]:
tasaRes = len(ddf[(ddf['respuesta']==1) & (ddf['estado_caso']=='caso resuelto')]
             )/len(ddf[(ddf['respuesta']==1) & (ddf['estado_caso']!='caso abierto')])*100
print ('Son resueltos %f %% de los casos' %tasaRes)

Son resueltos 50.704225 % de los casos


C. Indicar el mes del año pasado en el que Batman tuvo mayor participación en la investigación de casos.

In [6]:
ddf['mes']=ddf['fecha'].apply(lambda x:x.month)

In [7]:
ddf['anio']=ddf['fecha'].apply(lambda x:x.year)

In [8]:
ddf.head()

,fecha,id_caso,descripcion,estado_caso,categoria,latitud,longitud,respuesta,mes,anio
0,2018-01-08,100,asesinato,caso resuelto,1,212,4998,0,1,2018
1,2018-05-23,101,bandalismo,caso resuelto,4,179,4521,0,5,2018
2,2018-11-30,102,asesinato,caso resuelto,2,157,3973,0,11,2018
3,2018-08-17,103,bandalismo,cerrado sin resolución,5,356,4069,0,8,2018
4,2018-01-08,104,violacion,caso resuelto,5,327,3541,1,1,2018


In [9]:
mdf=ddf.groupby('anio').get_group(2019).groupby('mes').agg({'respuesta':'sum'}).sort_values(by='respuesta',ascending=False).reset_index()
mdf.head()

,mes,respuesta
0,7,27
1,10,27
2,6,24
3,11,22
4,5,21


In [10]:
meses=mdf[mdf['respuesta']==mdf.loc[0,'respuesta']].mes.values
meses

array([ 7, 10], dtype=int64)

In [11]:
print('Mes que tuvo mayor participación: %s' %','.join(map(str, meses)))

Mes que tuvo mayor participación: 7,10
